In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import time

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

---

## Part 1: CUDA Events Basics

### What are CUDA Events?

```
EVENT = A marker in a stream that can be:
        • Recorded (placed at a point in stream)
        • Queried (check if reached)
        • Waited on (block until reached)
        • Used for timing

Stream Timeline:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
[H2D Copy] → [event1] → [Kernel] → [event2] → [D2H Copy]
              ↑                      ↑
        record start           record stop
              └─────────────────────┘
                 elapsed time!
```

### 🔷 CUDA C++ Implementation (Primary)

This example demonstrates event creation, recording, synchronization, and timing.

In [ ]:
%%writefile event_basics.cu
// event_basics.cu - Event creation, recording, and synchronization
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void computeKernel(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        for (int i = 0; i < 100; i++) {
            data[tid] = sqrtf(data[tid] + 1.0f);
        }
    }
}

int main() {
    const int N = 1 << 22;
    
    float *d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    // ============================================
    // Event Creation
    // ============================================
    cudaEvent_t start, stop;
    
    // Default events (for timing and sync)
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Event with flags:
    // cudaEventDefault      - Default behavior
    // cudaEventBlockingSync - CPU blocks instead of spinning
    // cudaEventDisableTiming- Faster, but can't time
    // cudaEventInterprocess - For IPC
    
    cudaEvent_t fastEvent;
    cudaEventCreateWithFlags(&fastEvent, cudaEventDisableTiming);
    
    // ============================================
    // Event Recording (place marker in stream)
    // ============================================
    cudaEventRecord(start);  // Default stream
    // OR: cudaEventRecord(start, stream);  // Specific stream
    
    computeKernel<<<256, 256>>>(d_data, N);
    computeKernel<<<256, 256>>>(d_data, N);
    computeKernel<<<256, 256>>>(d_data, N);
    
    cudaEventRecord(stop);
    
    // ============================================
    // Event Synchronization
    // ============================================
    
    // Method 1: Block until event completes
    cudaEventSynchronize(stop);
    
    // Method 2: Query (non-blocking)
    cudaError_t status = cudaEventQuery(stop);
    if (status == cudaSuccess) {
        printf("Event completed!\n");
    } else if (status == cudaErrorNotReady) {
        printf("Event not yet reached\n");
    }
    
    // ============================================
    // Event Timing
    // ============================================
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Kernel time: %.3f ms\n", milliseconds);
    
    // Cleanup
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaEventDestroy(fastEvent);
    cudaFree(d_data);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o event_basics event_basics.cu
!./event_basics

---

## Part 2: Accurate GPU Timing

### Why Events for Timing?

```
CPU Timing (WRONG for GPU):
━━━━━━━━━━━━━━━━━━━━━━━━━━━
auto start = clock();
kernel<<<>>>(...);        // Returns immediately!
auto end = clock();       // Measures launch overhead only
// WRONG: Measures ~0.001ms instead of actual kernel time

GPU Event Timing (CORRECT):
━━━━━━━━━━━━━━━━━━━━━━━━━━
cudaEventRecord(start);
kernel<<<>>>(...);        // Runs asynchronously
cudaEventRecord(stop);
cudaEventSynchronize(stop);
cudaEventElapsedTime(&ms, start, stop);
// CORRECT: Measures actual GPU execution time
```

### 🔷 CUDA C++ Implementation (Primary)

This example demonstrates proper GPU timing techniques including warmup, single kernel timing, averaging over multiple iterations, and timing individual sections.

In [ ]:
%%writefile accurate_timing.cu
// accurate_timing.cu - Proper GPU timing
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void kernel(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        for (int i = 0; i < 1000; i++) {
            data[tid] = sinf(data[tid]);
        }
    }
}

int main() {
    const int N = 1 << 20;
    float *d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // ============================================
    // Warmup (IMPORTANT!)
    // ============================================
    // First kernel launch has overhead (JIT, caching, etc.)
    kernel<<<256, 256>>>(d_data, N);
    cudaDeviceSynchronize();
    
    // ============================================
    // Time Single Kernel
    // ============================================
    cudaEventRecord(start);
    kernel<<<256, 256>>>(d_data, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float singleTime;
    cudaEventElapsedTime(&singleTime, start, stop);
    printf("Single kernel: %.3f ms\n", singleTime);
    
    // ============================================
    // Time Multiple Iterations (more accurate)
    // ============================================
    const int ITERATIONS = 100;
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        kernel<<<256, 256>>>(d_data, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float totalTime;
    cudaEventElapsedTime(&totalTime, start, stop);
    printf("Average over %d runs: %.3f ms\n", 
           ITERATIONS, totalTime / ITERATIONS);
    
    // ============================================
    // Time Individual Sections
    // ============================================
    cudaEvent_t e1, e2, e3;
    cudaEventCreate(&e1);
    cudaEventCreate(&e2);
    cudaEventCreate(&e3);
    
    float *h_data;
    cudaMallocHost(&h_data, N * sizeof(float));
    
    cudaEventRecord(e1);
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaEventRecord(e2);
    kernel<<<256, 256>>>(d_data, N);
    cudaEventRecord(e3);
    cudaEventSynchronize(e3);
    
    float h2dTime, kernelTime;
    cudaEventElapsedTime(&h2dTime, e1, e2);
    cudaEventElapsedTime(&kernelTime, e2, e3);
    
    printf("H2D transfer: %.3f ms\n", h2dTime);
    printf("Kernel exec:  %.3f ms\n", kernelTime);
    
    // Cleanup
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaEventDestroy(e1);
    cudaEventDestroy(e2);
    cudaEventDestroy(e3);
    cudaFree(d_data);
    cudaFreeHost(h_data);
    
    return 0;
}

---

## Part 3: Inter-Stream Synchronization

### cudaStreamWaitEvent

```
Without Event Sync:            With Event Sync:
━━━━━━━━━━━━━━━━━━━━           ━━━━━━━━━━━━━━━━━━━
Stream A: [Produce Data]       Stream A: [Produce]──┐
Stream B: [Use Data] ← RACE!   Stream B:            └─→[Wait][Use]
                                                    ↑
                                              cudaStreamWaitEvent
```

### 🔷 CUDA C++ Implementation (Primary)

```cpp
// inter_stream_sync.cu - Synchronizing between streams
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void produce(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        data[tid] = (float)tid * 2.0f;  // Produce data
    }
}

__global__ void consume(const float* input, float* output, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        output[tid] = input[tid] + 1.0f;  // Use produced data
    }
}

int main() {
    const int N = 1 << 20;
    
    float *d_produced, *d_consumed;
    cudaMalloc(&d_produced, N * sizeof(float));
    cudaMalloc(&d_consumed, N * sizeof(float));
    
    cudaStream_t streamA, streamB;
    cudaStreamCreate(&streamA);
    cudaStreamCreate(&streamB);
    
    // Event for synchronization
    cudaEvent_t dataReady;
    cudaEventCreate(&dataReady);
    
    // ============================================
    // Stream A: Produce data
    // ============================================
    produce<<<256, 256, 0, streamA>>>(d_produced, N);
    
    // Record event when production is done
    cudaEventRecord(dataReady, streamA);
    
    // ============================================
    // Stream B: Wait for data, then consume
    // ============================================
    // This makes streamB wait until streamA reaches dataReady
    cudaStreamWaitEvent(streamB, dataReady);
    
    consume<<<256, 256, 0, streamB>>>(d_produced, d_consumed, N);
    
    // Both streams can do other independent work before/after
    
    cudaDeviceSynchronize();
    printf("Producer-consumer with event sync complete!\n");
    
    // ============================================
    // Complex Dependency Graph
    // ============================================
    //       A1
    //      /  \
    //     B1  B2
    //      \  /
    //       C1
    
    cudaStream_t sA, sB1, sB2, sC;
    cudaStreamCreate(&sA);
    cudaStreamCreate(&sB1);
    cudaStreamCreate(&sB2);
    cudaStreamCreate(&sC);
    
    cudaEvent_t afterA, afterB1, afterB2;
    cudaEventCreate(&afterA);
    cudaEventCreate(&afterB1);
    cudaEventCreate(&afterB2);
    
    // A1: Initial work
    produce<<<256, 256, 0, sA>>>(d_produced, N);
    cudaEventRecord(afterA, sA);
    
    // B1 and B2: Parallel work after A
    cudaStreamWaitEvent(sB1, afterA);
    cudaStreamWaitEvent(sB2, afterA);
    consume<<<256, 256, 0, sB1>>>(d_produced, d_consumed, N/2);
    consume<<<256, 256, 0, sB2>>>(d_produced + N/2, d_consumed + N/2, N/2);
    cudaEventRecord(afterB1, sB1);
    cudaEventRecord(afterB2, sB2);
    
    // C1: After both B1 and B2 complete
    cudaStreamWaitEvent(sC, afterB1);
    cudaStreamWaitEvent(sC, afterB2);
    produce<<<256, 256, 0, sC>>>(d_consumed, N);  // Final processing
    
    cudaDeviceSynchronize();
    printf("Complex dependency graph complete!\n");
    
    // Cleanup
    cudaEventDestroy(dataReady);
    cudaEventDestroy(afterA);
    cudaEventDestroy(afterB1);
    cudaEventDestroy(afterB2);
    cudaStreamDestroy(streamA);
    cudaStreamDestroy(streamB);
    cudaStreamDestroy(sA);
    cudaStreamDestroy(sB1);
    cudaStreamDestroy(sB2);
    cudaStreamDestroy(sC);
    cudaFree(d_produced);
    cudaFree(d_consumed);
    
    return 0;
}
```

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
# Python/Numba Event Timing Demo (OPTIONAL)

@cuda.jit
def heavy_kernel(data):
    tid = cuda.grid(1)
    if tid < data.shape[0]:
        val = data[tid]
        for _ in range(1000):
            val = val * 1.0001 + 0.0001
        data[tid] = val

n = 1 << 20

# Create device array
d_data = cuda.device_array(n, dtype=np.float32)

# Warmup
heavy_kernel[(n+255)//256, 256](d_data)
cuda.synchronize()

# Time with events
stream = cuda.stream()
start_event = stream.record()

for _ in range(10):
    heavy_kernel[(n+255)//256, 256, stream](d_data)

end_event = stream.record()
stream.synchronize()

elapsed = cuda.event_elapsed_time(start_event, end_event)
print(f"10 kernels: {elapsed:.2f} ms")
print(f"Per kernel: {elapsed/10:.2f} ms")

---

## Part 4: Event Flags & Advanced Usage

### Event Flags

```cpp
// event_flags.cu - Different event types

// cudaEventDefault - Standard event for timing and sync
cudaEvent_t defaultEvent;
cudaEventCreate(&defaultEvent);

// cudaEventBlockingSync - CPU thread yields instead of spinning
// Better for power efficiency, slightly higher latency
cudaEvent_t blockingEvent;
cudaEventCreateWithFlags(&blockingEvent, cudaEventBlockingSync);

// cudaEventDisableTiming - Can't use for timing, but faster
// Good for pure synchronization
cudaEvent_t syncOnlyEvent;
cudaEventCreateWithFlags(&syncOnlyEvent, cudaEventDisableTiming);

// Combine flags
cudaEvent_t fastSyncEvent;
cudaEventCreateWithFlags(&fastSyncEvent, 
                          cudaEventBlockingSync | cudaEventDisableTiming);
```

### When to Use Each

```
┌─────────────────────────────────────────────────────────┐
│                    EVENT FLAGS                          │
├──────────────────────┬──────────────────────────────────┤
│ Flag                 │ Use Case                         │
├──────────────────────┼──────────────────────────────────┤
│ Default              │ Timing + sync, general use       │
│ BlockingSync         │ Long waits, power-sensitive      │
│ DisableTiming        │ Pure sync, maximum performance   │
│ Both combined        │ Sync-only, power-efficient       │
└──────────────────────┴──────────────────────────────────┘
```

---

## Exercises

### Exercise 1: Benchmark Kernel
Create a timing utility that:
- Warms up the kernel
- Runs multiple iterations
- Reports min/max/avg times

### Exercise 2: DAG Executor
```cpp
// Implement a simple DAG (Directed Acyclic Graph) of kernels:
//     A
//    /|\
//   B C D
//    \|/
//     E
```

### Exercise 3: Event Pool
Implement an event pool that reuses events to avoid creation overhead.

---

## Key Takeaways

```
┌─────────────────────────────────────────────────────────┐
│                 CUDA EVENTS                             │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Event Lifecycle:                                       │
│  • cudaEventCreate(&event)                              │
│  • cudaEventRecord(event, stream)                       │
│  • cudaEventSynchronize(event)                          │
│  • cudaEventElapsedTime(&ms, start, stop)               │
│  • cudaEventDestroy(event)                              │
│                                                         │
│  Inter-Stream Sync:                                     │
│  • cudaStreamWaitEvent(stream, event)                   │
│  • Creates dependency: stream waits for event           │
│                                                         │
│  Timing Best Practices:                                 │
│  • Warmup before timing                                 │
│  • Multiple iterations for accuracy                     │
│  • Use events, not CPU timers                           │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

## Week 9 Complete!

**Day 5:** Practice exercises and checkpoint quiz.

**Next Week:** CUDA Graphs - compile entire workflows for minimal overhead!